In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [ ]:
for item in dataset:
  print(item)
  break

tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
dataset = dataset.repeat(3).batch(7) # drop_remainder=True отбросит последний пакет(неполный)

for item in dataset:
  print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [ ]:
dataset = dataset.apply(tf.data.experimental.unbatch())

for item in dataset:
  print(item, end=' ')

dataset = dataset.filter(lambda x: x < 10)

print('\nFiltered')

for item in dataset.take(3):
  print(item)

tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(2, shape=(), dtype=int32) tf.Tensor(3, shape=(), dtype=int32) tf.Tensor(4, shape=(), dtype=int32) tf.Tensor(5, shape=(), dtype=int32) tf.Tensor(6, shape=(), dtype=int32) tf.Tensor(7, shape=(), dtype=int32) tf.Tensor(8, shape=(), dtype=int32) tf.Tensor(9, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(2, shape=(), dtype=int32) tf.Tensor(3, shape=(), dtype=int32) tf.Tensor(4, shape=(), dtype=int32) tf.Tensor(5, shape=(), dtype=int32) tf.Tensor(6, shape=(), dtype=int32) tf.Tensor(7, shape=(), dtype=int32) tf.Tensor(8, shape=(), dtype=int32) tf.Tensor(9, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(2, shape=(), dtype=int32) tf.Tensor(3, shape=(), dtype=int32) tf.Tensor(4, shape=(), dtype=int32) tf.Tensor(5, shape=(), dtype=int32) tf.Tensor(6, shape=(), dtype=int32) tf.Tensor(7, shape=(), dtype

In [ ]:
dataset = tf.data.Dataset.range(10).repeat(3) # от 0 до 9, три раза
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)

for item in dataset:
  print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


**Команда shuf в Linux перемешает строки в файле**

```
shuf file.txt or shuf file.txt -o shuffled_file.txt
```

# Загрузка и предварительная обработка данных из множества файлов CSV

prefetch(1) помогает подготавливать данные на CPU, а обучать на GPU. Тем самым ускоряя подачу данных для обучения

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, n_read_threads=None, shuffle_buffer_size=1e4, n_parse_threads=5, batch_size=32):
  dataset = tf.data.Dataset.list_files(filepaths)
  # Соединяем файлы для перемешивания
  dataset = dataset.interleave(
      lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
      cycle_length=n_readers, num_parallel_calls=n_parse_threads
  )
  dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
  dataset = dataset.shuflle(shuffle_buffer_size).repeat(repeat)
  return dataset.batch(batchsize).prefetch(1)

Обучение входного конвейера с API Data

In [ ]:
@tf.function
def train(model, opimizer, loss_fn, n_epochs, [...]):
  train_set = csv_reader_dataset(train_filepaths, repeat=n_epochs, [...])
  for X_batch, y_batch in train_set:
    with tf.GradientTape() as tape:
      y_pred = model(X_batch)
      main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
      loss = tf.add_n([main_loss] + model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

TF Record - Формат в TensorFlow для хранения очень крупных объемов данных и их эффективного чтения.

Использует CRC для проверки повреждения

In [ ]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
  f.write(b"This is the first record")
  f.write(b"And this is the second record")

tf.data.TFRecordDataset - чтение одного и более файлов TFRecord

In [ ]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
  print(item)

One-Hot Encoding - немного категориальных признаков

Embedding - много категориальных признаков

In [ ]:
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND"])
cat_indices = take.lookup(categories)
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + 2) # + 2 т.к всего 5 признаков

In [ ]:
embedding_dim = 2 # двумерные вложения
embed_init = tf.random.uniform([len(vocab) + 2, embedding_dim]) # матрица [n, dim] где n количество признаков
embedding_matrix = tf.Variable(embed_init) # пример [0.654, 0.443] * 5(random)

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND"])
cat_indices = take.lookup(categories)
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

In [ ]:
embedding = keras.layers.Embedding(input_dim=len(vocab) + 2, output_dim=embedding_dim)
embedding(cat_indices)